# Mandelbrot Using Workers

It is acknowleged that Python is typically *not* ideal for running calculation intensive workloads.

The purpose is to demonstrate `Workers` running python code across a network, and to show that performance/timing numbers can be improved upon when partitioning work across multiple machines (assuming the overhead of communication, etc. is outweigh by the processing time).

Load PENVM setup for deployed release 0.1.0.

In [ ]:
%run "penvm-0.1.0-setup.ipynb"

Set up World and boot "default" network.

In [ ]:
world = World(filename="world-4-local.penvm")
network = world.get_network()
network.boot()
nworkers = len(network.get_machines())

Add code.

In [ ]:
#! /usr/bin/env python3
#
# mandelbrot.py

import matplotlib.pyplot as plt
import numpy as np
import time

from penvm.ext.workers import PythonCodeWorker

import mandelbrotlib

np.warnings.filterwarnings("ignore")


def show(stability):
    plt.imshow(stability, cmap="plasma")  # 20
    plt.gca().set_aspect("equal")
    plt.axis("off")
    plt.tight_layout()
    plt.show()


class MandelbrotWorker(PythonCodeWorker):
    
    def combine(self, results):
        return results[0] if self.nworkers == 1 else np.concatenate(results)

    def partition(self, fnname, xmin, xmax, ymin, ymax, density, niterations, z, **kwargs):
        # _density = density / nworkers
        # vertical strips; 31-10-2014   2no change in density
        _density = density
        ydiff = ymax - ymin
        ystep = ydiff / self.nworkers
        for i in range(self.nworkers):
            _ymin = ymin + i * ystep
            _ymax = ymin + (i + 1) * ystep
            yield ((fnname, xmin, xmax, _ymin, _ymax, _density, niterations, z), kwargs)

    def use_fallback(self, *args, **kwargs):
        # TODO: check against density and niterations
        return False

w = MandelbrotWorker(
    mandelbrotlib.calculate,
    open("mandelbrotlib.py").read(),
    network,
    nworkers,
    collect_response=True,
)

run = w.run
wrun = w.wrun("calculate")

Note:

* `w` is a `MandelbrotWorker` instance.
* `run` is the `MandelbrotWorker.run` instance method.
* `wrun` is a wrapped `MandelbrotWorker.run` instance method with the `fnname` provided.

Whereas `run` requires explicit specification of a function name, `wrun` allows for easy, drop-in use of a client-side function name for a function that can take the expected `args` and `kwargs`.

Run with explicit function name, and plot results

In [ ]:
result = run("calculate", -2, 0.5, -1.5, 1.5, 256, 100, 0)
show(result.clip(0, 2))

Run with wrapped function name, and plot results.

In [ ]:
result = wrun(-2, 0.5, -1.5, 1.5, 256, 100, 0)
show(result.clip(0, 2))

Run again with other settings. Provide elapsed time, too.

In [ ]:
t0 = time.time()
result = wrun(-2, 0.5, -1.5, 1.5, 256, 100, 0)
print(f"elapsed ({time.time()-t0})")
show(result.clip(0, 2))

In [ ]:
t0 = time.time()
result = wrun(-2, 0.5, -1.5, 1.5, 512, 200, 0)
print(f"elapsed ({time.time()-t0})")
show(result.clip(0, 2))

Run with different `nworkers` values (with appropriate settings).

In [ ]:
for i in range(1, 5):
    w._nworkers = i
    t0 = time.time()
    result = wrun(-2, 0.5, -1.5, 1.5, 768, 200, 0)
    print(f"nworkers ({i}) elapsed ({time.time()-t0})")
    show(result.clip(0, 2))

Clean up.

In [ ]:
world.shutdown()